In [73]:
import pandas as pd

## here's a method to get the PKL path for a random experiment

In [71]:
def get_random_pkl_path(random_state=0):
    from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc
    from visual_behavior import database as db
    data_storage_directory = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/production_cache'

    cache = bpc.from_s3_cache(cache_dir=data_storage_directory)
    experiment_table = cache.get_ophys_experiment_table()
    bsid = experiment_table.sample(random_state = random_state)['behavior_session_id'].iloc[0]
    pkl_path = db.get_pkl_path(session_id = bsid)
    return pkl_path

## the session UUID aka foraging_id is located at the top level of the pkl file dictionary in a key called 'session_uuid':
Here it is for one random session

In [79]:
data = pd.read_pickle(get_random_pkl_path(random_state=0))
print(data.keys())
print(data['session_uuid'])

dict_keys(['unpickleable', 'items', 'start_time', 'script', 'threads', 'stop_time', 'session_uuid', 'platform_info'])
070af869-f96c-412b-bc1e-bc8801a34562


And here it is for another random session

In [87]:
data = pd.read_pickle(get_random_pkl_path(random_state=1))
print(data.keys())
print(data['session_uuid'])

dict_keys(['comp_id', 'unpickleable', 'items', 'start_time', 'script', 'rig_id', 'threads', 'stop_time', 'session_uuid', 'platform_info'])
f4a89a74-45f1-4421-966b-df7d92756cef


In [88]:
type(data['session_uuid'])

str

## But this key appears to be missing in a path that Wayne provided to me:

In [89]:
wayne_path = '/allen/programs/braintv/production/visualbehavior/prod3/specimen_912560523/ophys_session_965063340/965063340.pkl'
data = pd.read_pickle(wayne_path)
print(data.keys())
print(data['session_uuid'])

dict_keys(['script', 'items', 'start_time', 'unpickleable', 'threads', 'stop_time', 'platform_info'])


KeyError: 'session_uuid'

### For the errant session, here is the contents of 'platform_info'

In [84]:
for key, value in data['platform_info'].items():
    print(key, value)

hardware ('Intel64 Family 6 Model 63 Stepping 2, GenuineIntel', 'AMD64')
camstim 0.5.6+g9c5966f.b2062
pyglet 1.2.4
computer_name MESO1STIM
opengl 4.5.0 NVIDIA 384.76
python 2.7.15
rig_id unknown
os ('Windows', '10', '10.0.14393')
psychopy 1.82.02
camstim_git_hash None


### For the errant session, here is the contents of data['items']['behavior']['params']

In [86]:
for key, value in data['items']['behavior']['params'].items():
    print(key, value)

reward_volume 0.007
auto_reward_delay 0.15
change_time_scale 0.3
failure_repeats 5
start_stop_padding 300.0
end_after_response True
periodic_flash [0.25, 0.5]
flash_omit_probability 0.05
change_flashes_max 12
mouse_id 477052
change_time_dist geometric
stimulus_window 6.0
user_id sams
response_window [0.15, 0.75]
change_flashes_min 4
catch_frequency None
stage OPHYS_2_images_G_passive
auto_reward_vol 0.005
min_no_lick_time 0.0
timeout_duration 0.3
free_reward_trials 10000
task_id DoC
stimulus {'params': {'image_set': '//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/image_dictionaries/Natural_Images_Lum_Matched_set_ophys_G_2019.05.26.pkl', 'sampling': 'even'}, 'class': 'images', 'luminance_matching_intensity': -0.46}
epilogue {'params': {'frame_length': 0.03333333333333333, 'movie_path': '//allen/programs/braintv/workgroups/neuralcoding/Saskia/Visual Stimuli 151207/Movie_TOE1.npy', 'runs': 10, 'size': [1920, 1080]}, 'name': 'fingerprint'}
volume_limit 5.0
end_after_response_

## I don't know why this key would be missing in this file. Is this the only case where this is true? I believe the UUID is assigned by camstim at the end of the session. Could this session have crashed before the UUID assignment function was called?

## I think this is a question for MPE

## According to Wayne, the UUID should be '11599422-bcfe-4a8c-bd3c-4f08f5d9075d', I'm going to write that to data, then save out as a new PKL file

In [95]:
import os
import pandas as pd

wayne_path = '/allen/programs/braintv/production/visualbehavior/prod3/specimen_912560523/ophys_session_965063340/965063340.pkl'
data = pd.read_pickle(wayne_path)

data['session_uuid'] = '11599422-bcfe-4a8c-bd3c-4f08f5d9075d'

folder, filename = os.path.split(wayne_path)
new_filename = filename.replace('.pkl', '_edited.pkl')
new_folder = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior'
new_savepath = os.path.join(new_folder, new_filename)
pd.to_pickle(data, new_savepath)
print('saved to {}'.format(new_savepath))

saved to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/965063340_edited.pkl


'/allen/programs/braintv/production/visualbehavior/prod3/specimen_912560523/ophys_session_965063340/965063340_edited.pkl'